In [29]:
#ライブラリをインポート
import os
import re
import sys
import csv
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [30]:
#定数を定義
BINS = 10000  #ヒストグラムのビンの数
EPSILON = .00001  #スムージングパラメータ
UPPER_LIMIT = 1.1 #静止区間の上限
LOWER_LIMIT = 0.9 #静止区間の加減
STATIONARY_INTERVALS = 5  #静止区間除去のサンプルの間隔(静止区間が何サンプル連続したら除去するか)

In [31]:
#ディレクトリ内のデータセットのファイル名と周波数を取得する関数
def get_filename_and_Hz(path: str) -> list[int, str]:
    filename = os.listdir(path) #引数のパスのディレクトリの中のファイル名一覧を取得
    filename_and_Hz=[]  #ファイル名と周波数を格納するリストを宣言

    for file in filename:
        Hz = re.search(r'\d+', file)    #正規表現を用いてファイル名の中で一番最初に出てくる数字(周波数)を取得
        if Hz:  #数字の入っていないファイル名があるとエラーを吐くので、このif文でチェックする
            filename_and_Hz.append([int(Hz.group(0)), file])    #ファイル名と周波数を格納

    return filename_and_Hz

In [32]:
#ファイル名と周波数を分けて出力する関数
def divide_filename_and_Hz(filename_and_Hz: list[int, str]) -> tuple[list[int], list[str]]:
    Hz = []
    filename = []
    for row in filename_and_Hz:
      Hz.append(row[0])
      filename.append(row[1])

    return Hz, filename

In [33]:
#加速度データのCSVファイルから3軸加速度を取得する関数
def get_acceleration(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[2]))
            AccY.append(float(row[3]))
            AccZ.append(float(row[4]))

    return AccX, AccY, AccZ

In [34]:
#静止区間を除去する関数
def remove_stationary_intervals(AccX: list[float], AccY: list[float], AccZ: list[float]) -> list[float]:
    #各軸の加速度の平均を求める
    AvgAccX = sum(AccX) / len(AccX)
    AvgAccY = sum(AccY) / len(AccY)
    AvgAccZ = sum(AccZ) / len(AccZ)

    AvgResultantAcc = math.sqrt(AvgAccX ** 2 + AvgAccY ** 2 + AvgAccZ ** 2) #重力加速度の推定値=合成加速度の平均を求める

    ResultantAcc = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(AccX, AccY, AccZ)]   #各時刻の合成加速度を求める

    #各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が一定以上以上連続している区間)を除去する
    i=0 #ループ変数
    counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
    while i < len(ResultantAcc):
        if AvgResultantAcc * LOWER_LIMIT < ResultantAcc[i] < AvgResultantAcc * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
            counter += 1    #範囲内ならカウントを増やす
            if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
                del ResultantAcc[i+1-STATIONARY_INTERVALS:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書くとstart <= x < stopの範囲が選択される #start番目の値は含まれるがstop番目の値は含まれない
                counter = 0 #カウンターをリセット
                i -= STATIONARY_INTERVALS   #削除した分インデックスがズレるので補正する
        else:
            counter = 0 #カウンターをリセット
        i += 1

    return ResultantAcc  #静止区間を除去した後のリストを返す

In [35]:
#連続する2サンプルの差分を取る関数
def differences_of_acceleration(ResultantAcc: list[float]) -> list[float]:
    index = 0
    DifferenceAcc = []
    for dif in ResultantAcc[:-1]:
        DifferenceAcc.append(math.fabs(ResultantAcc[index + 1]*100000 - ResultantAcc[index]*100000))    #100000倍して誤差を取る
        index += 1

    return DifferenceAcc

In [36]:
#KLダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def KL_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #KLダイバージェンスの値を返す
    return np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, b_hist)])

In [37]:
#JSダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def JS_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #2つの分布の平均値を求める
    mean_hist = (a_hist + b_hist) / 2.0

    #平均とそれぞれの分布のKLダイバージェンスを算出
    kl_a = np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, mean_hist)])
    kl_b = np.sum([ai * np.log(ai / bi) for ai, bi in zip(b_hist, mean_hist)])

    #JSダイバージェンスの値を返す
    return (kl_a + kl_b) / 2.0

In [38]:
#データフレームの各行の中で2番目に小さい値が格納されている場所を調べる関数(最小値は同じ確率分布同士の0.0)
def get_index_and_columns_of_second_smallest(df: pd.DataFrame) -> list[str, str]:
    index_and_columns_of_second_smallest = []  #データフレームの中で2番目に小さい値が格納されている場所のインデックス名とカラム名を格納する変数
    for i in range(len(df)):
        sorted_row = df.iloc[i].sort_values()   #.ilocでデータフレームの要素を行、列の番号の添字で指定する    #各行の要素を昇順に並び替える
        second_smallest_columns = sorted_row.index[1] #各行の2番目に小さい値が格納されているカラム[1]の名前を取得
        #second_smallest_label = df.columns.get_loc(second_smallest_index)
        index_and_columns_of_second_smallest.append((df.index[i], second_smallest_columns))    #インデックスとカラムのラベル名の組を二次元配列に追加
    return index_and_columns_of_second_smallest

In [39]:
#推定精度を算出する巻数
def calculate_accuracy(index_and_columns_of_second_smallest: list[str, str]) -> float:
    counter = 0
    for i in range(len(index_and_columns_of_second_smallest)):
        #インデックスとカラムのラベル名が同じならばカウンターを1増やす
        if index_and_columns_of_second_smallest[i][0] == index_and_columns_of_second_smallest[i][1]:
            counter += 1

    return (counter / len(index_and_columns_of_second_smallest)) * 100  #精度を100分率で返す

In [40]:
#入力された加速度の差分のリストからヒストグラムを作る関数
def create_histogram(DifferenceAcc_list: list[float]) -> np.histogram:
    min_value = min(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も小さい数
    max_value = max(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も大きい数

    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)
    print(DifferenceAcc_hist[0])

    for i in range(len(DifferenceAcc_list)):
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
        DifferenceAcc_hist[i] = (DifferenceAcc_hist[i] + EPSILON) / DifferenceAcc_hist[i].sum()     #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    return DifferenceAcc_hist

In [41]:
#KLダイバージェンスとJSダイバージェンス算出の一連の流れを自動化
def KL_and_JS(path: str):
    filename_and_Hz = get_filename_and_Hz(path)
    filename_and_Hz.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_filename_and_Hz(filename_and_Hz)
    Hz = [str(hz) + "Hz" for hz in Hz]  #周波数の値+"Hz"のリストを作りデータフレームのラベルに用いる

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []
    resultKLD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    resultJSD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化


    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

    #KLダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultKLD[i][j] = KL_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #JSダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultJSD[i][j] = JS_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #結果を出力
    df_KLD = pd.DataFrame(resultKLD, index=Hz, columns=Hz)
    display(df_KLD)
    print(f"KLダイバージェンスによる推定精度は{calculate_accuracy(get_index_and_columns_of_second_smallest(df_KLD))}%です")

    df_JSD = pd.DataFrame(resultJSD, index=Hz, columns=Hz)
    display(df_JSD)
    print(f"JSダイバージェンスによる推定精度は{calculate_accuracy(get_index_and_columns_of_second_smallest(df_JSD))}%です")

In [ ]:
def bunrui(path: str):
    filename_and_Hz = get_filename_and_Hz(path)
    filename_and_Hz.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_filename_and_Hz(filename_and_Hz)
    Hz = [str(hz) + "Hz" for hz in Hz]  #周波数の値+"Hz"のリストを作りデータフレームのラベルに用いる

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

    DifferenceAcc_hist = create_histogram(DifferenceAcc_list)
    x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, test_size = 0.2, train_size = 0.8, shuffle = True)

    # 学習する
    clf = SVC()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("正解率 = ", accuracy_score(y_test, y_pred))

In [42]:
KL_and_JS("data/")

,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,50Hz,50Hz,50Hz,...,50Hz,50Hz,50Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.165483,0.349133,0.391047,0.335301,0.178283,0.213432,0.162186,0.088181,0.385054,...,0.404467,0.282905,0.315165,1.174630,1.796806,0.550216,0.225328,1.632737,1.593108,1.346018
100Hz,0.101292,0.000000,0.335277,0.362438,0.323632,0.239413,0.206996,0.306338,0.080218,0.665483,...,0.644037,0.503558,0.538758,1.380534,2.247759,0.741282,0.291562,2.118427,2.022862,1.765751
100Hz,0.131331,0.253307,0.000000,0.142298,0.027826,0.063785,0.056752,0.285203,0.224845,0.298766,...,0.290547,0.254333,0.258173,1.584989,2.499185,0.760177,0.465417,1.866039,6.221170,1.632231
100Hz,0.149467,0.279472,0.060716,0.000000,0.035654,0.086962,0.077647,0.305933,0.248401,0.302698,...,0.311769,0.283433,0.279341,1.602716,2.476670,0.783450,0.483655,1.890320,5.816405,1.653716
100Hz,0.144172,0.269129,0.031389,0.052569,0.000000,0.080518,0.068941,0.308851,0.243945,0.284016,...,0.306553,0.283292,0.284119,1.608224,2.516639,0.789007,0.484465,1.904839,3.469898,1.629254
100Hz,0.063701,0.208967,0.120977,0.202100,0.146167,0.000000,0.083106,0.146979,0.124524,0.175910,...,0.189648,0.147443,0.159362,1.377196,2.207195,0.580913,0.300578,1.599983,2.443566,1.348680
100Hz,0.066090,0.172677,0.105659,0.164988,0.116378,0.066816,0.000000,0.191941,0.128972,0.268268,...,0.274890,0.223971,0.234436,1.410532,2.299417,0.639878,0.328186,1.745890,3.054139,1.471423
50Hz,0.251245,0.508458,0.940559,1.102883,0.938669,0.512539,0.712816,0.000000,0.217023,0.435024,...,0.549374,0.310513,0.392716,0.884118,1.382691,0.367208,0.184393,1.123058,1.244457,0.909697
50Hz,0.080040,0.141232,0.524343,0.595302,0.502241,0.262132,0.325707,0.150254,0.000000,0.482143,...,0.520806,0.335216,0.386619,1.086798,1.796402,0.499660,0.156621,1.605892,1.667787,1.320576
50Hz,0.254382,0.579850,0.565931,0.897258,0.484493,0.204042,0.356186,0.178765,0.330187,0.000000,...,0.091473,0.100231,0.089009,1.268222,1.807837,0.560574,0.397188,1.282759,5.469988,1.131140


KLダイバージェンスによる推定精度は80.95238095238095%です


,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,50Hz,50Hz,50Hz,...,50Hz,50Hz,50Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.023402,0.034959,0.039794,0.038754,0.015319,0.015382,0.032255,0.010084,0.065090,...,0.066186,0.047952,0.051757,0.293556,0.375917,0.145045,0.054821,0.343926,0.333888,0.305825
100Hz,0.023402,0.000000,0.063983,0.069266,0.066354,0.050463,0.040223,0.077904,0.019119,0.140029,...,0.137428,0.111981,0.118108,0.341493,0.436061,0.197184,0.082355,0.412623,0.400164,0.374434
100Hz,0.034959,0.063983,0.000000,0.010637,0.005026,0.016503,0.012710,0.079576,0.058973,0.077955,...,0.077091,0.071436,0.071347,0.374462,0.446642,0.210292,0.126113,0.418980,0.459662,0.382939
100Hz,0.039794,0.069266,0.010637,0.000000,0.005303,0.021798,0.017137,0.086782,0.065276,0.081824,...,0.083930,0.081050,0.078469,0.382020,0.452996,0.217578,0.131961,0.428494,0.461870,0.393578
100Hz,0.038754,0.066354,0.005026,0.005303,0.000000,0.020580,0.015461,0.088694,0.064457,0.076156,...,0.083405,0.081709,0.080663,0.384328,0.457323,0.220854,0.133469,0.432537,0.432942,0.391988
100Hz,0.015319,0.050463,0.016503,0.021798,0.020580,0.000000,0.008273,0.040528,0.031788,0.038523,...,0.039505,0.035289,0.036301,0.330734,0.404671,0.165833,0.083928,0.367968,0.367073,0.329927
100Hz,0.015382,0.040223,0.012710,0.017137,0.015461,0.008273,0.000000,0.052614,0.031881,0.059669,...,0.061052,0.053902,0.054720,0.345213,0.421411,0.180360,0.091085,0.390891,0.390101,0.351341
50Hz,0.032255,0.077904,0.079576,0.086782,0.088694,0.040528,0.052614,0.000000,0.029996,0.047085,...,0.049401,0.022535,0.028527,0.224406,0.293317,0.090016,0.032591,0.238689,0.241423,0.211167
50Hz,0.010084,0.019119,0.058973,0.065276,0.064457,0.031788,0.031881,0.029996,0.000000,0.084485,...,0.085444,0.056838,0.063215,0.273765,0.359728,0.132170,0.039311,0.320267,0.315900,0.286782
50Hz,0.065090,0.140029,0.077955,0.081824,0.076156,0.038523,0.059669,0.047085,0.084485,0.000000,...,0.008875,0.020241,0.015647,0.310326,0.360912,0.154268,0.105647,0.313434,0.381111,0.288024


JSダイバージェンスによる推定精度は80.95238095238095%です


In [16]:
#データセットと周波数を読み込む
path = "data/"  #データセットのあるディレクトリまでのパス
filename_and_Hz = get_filename_and_Hz(path)
filename_and_Hz.sort(reverse=True)  #周波数の大きい順にソート
Hz, filename = divide_filename_and_Hz(filename_and_Hz)

#使う変数を宣言
AccX, AccY, AccZ = [], [], []
ResultantAcc = []
DifferenceAcc_list = []
resultKLD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
resultJSD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化

#各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
for i in filename:
    AccX, AccY, AccZ = get_acceleration(path+i)
    ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
    DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

In [17]:
#KLダイバージェンスの値を格納
for i in range(len(filename)):
    for j in range(len(filename)):
        resultKLD[i][j] = KL_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

for i in range(len(filename)):
    for j in range(len(filename)):
        resultJSD[i][j] = JS_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

In [18]:
#indexとcolumnsのラベル名についてのテストコード
#Zh = ["a","b","c","d","e","f","g","h","i","j","k","l","m"]
#df = pd.DataFrame(resultKLD, index=Hz, columns=Zh) #とりあえず残している

#KLダイバージェンスの結果を格納するデータフレームの定義
HzHz = [str(hz) + "Hz" for hz in Hz]

In [19]:
df_KLD = pd.DataFrame(resultKLD, index=HzHz, columns=HzHz)
display(df_KLD)
print(f"KLダイバージェンスによる推定精度は{calculate_accuracy(get_index_and_columns_of_second_smallest(df_KLD))}%です")

df_JSD = pd.DataFrame(resultJSD, index=HzHz, columns=HzHz)
display(df_JSD)
print(f"JSダイバージェンスによる推定精度は{calculate_accuracy(get_index_and_columns_of_second_smallest(df_JSD))}%です")

,100Hz,100Hz,100Hz,100Hz,100Hz,50Hz,50Hz,50Hz,50Hz,50Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.142298,0.027826,0.063785,0.056752,0.298766,0.419160,0.290547,0.254333,0.258173,1.866039,6.221170,1.632231
100Hz,0.060716,0.000000,0.035654,0.086962,0.077647,0.302698,0.387250,0.311769,0.283433,0.279341,1.890320,5.816405,1.653716
100Hz,0.031389,0.052569,0.000000,0.080518,0.068941,0.284016,0.332158,0.306553,0.283292,0.284119,1.904839,3.469898,1.629254
100Hz,0.120977,0.202100,0.146167,0.000000,0.083106,0.175910,0.228363,0.189648,0.147443,0.159362,1.599983,2.443566,1.348680
100Hz,0.105659,0.164988,0.116378,0.066816,0.000000,0.268268,0.316322,0.274890,0.223971,0.234436,1.745890,3.054139,1.471423
50Hz,0.565931,0.897258,0.484493,0.204042,0.356186,0.000000,0.445812,0.091473,0.100231,0.089009,1.282759,5.469988,1.131140
50Hz,0.618447,0.992268,0.524416,0.212756,0.360928,0.185423,0.000000,0.082131,0.113101,0.098568,1.299759,5.408781,1.167835
50Hz,0.414558,0.658042,0.493554,0.184383,0.312063,0.045998,0.106933,0.000000,0.085598,0.073647,1.297233,3.000684,1.115154
50Hz,0.566095,0.960066,0.669918,0.256241,0.456115,0.139010,0.351962,0.203228,0.000000,0.115491,1.135400,2.657059,0.970256
50Hz,0.523132,0.740423,0.604702,0.252626,0.403401,0.107543,0.248496,0.150248,0.086668,0.000000,1.161516,1.577764,0.977447


KLダイバージェンスによる推定精度は100.0%です


,100Hz,100Hz,100Hz,100Hz,100Hz,50Hz,50Hz,50Hz,50Hz,50Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.010637,0.005026,0.016503,0.012710,0.077955,0.095592,0.077091,0.071436,0.071347,0.418980,0.459662,0.382939
100Hz,0.010637,0.000000,0.005303,0.021798,0.017137,0.081824,0.096097,0.083930,0.081050,0.078469,0.428494,0.461870,0.393578
100Hz,0.005026,0.005303,0.000000,0.020580,0.015461,0.076156,0.087702,0.083405,0.081709,0.080663,0.432537,0.432942,0.391988
100Hz,0.016503,0.021798,0.020580,0.000000,0.008273,0.038523,0.044950,0.039505,0.035289,0.036301,0.367968,0.367073,0.329927
100Hz,0.012710,0.017137,0.015461,0.008273,0.000000,0.059669,0.066984,0.061052,0.053902,0.054720,0.390891,0.390101,0.351341
50Hz,0.077955,0.081824,0.076156,0.038523,0.059669,0.000000,0.034984,0.008875,0.020241,0.015647,0.313434,0.381111,0.288024
50Hz,0.095592,0.096097,0.087702,0.044950,0.066984,0.034984,0.000000,0.011727,0.027491,0.020202,0.326985,0.393367,0.303382
50Hz,0.077091,0.083930,0.083405,0.039505,0.061052,0.008875,0.011727,0.000000,0.020092,0.015954,0.326493,0.341439,0.294369
50Hz,0.071436,0.081050,0.081709,0.035289,0.053902,0.020241,0.027491,0.020092,0.000000,0.010519,0.276386,0.299422,0.251553
50Hz,0.071347,0.078469,0.080663,0.036301,0.054720,0.015647,0.020202,0.015954,0.010519,0.000000,0.285147,0.287661,0.256646


JSダイバージェンスによる推定精度は100.0%です


In [22]:
DifferenceAcc_hist = create_histogram(DifferenceAcc_list)
type(DifferenceAcc_hist)

[0. 0. 0. ... 0. 0. 0.]


numpy.ndarray

In [28]:
DifferenceAcc_hist[3].sum()

1.0014705882352943

In [ ]:
len(DifferenceAcc_hist)

In [ ]:
model = LinearRegression()  # 線形回帰を使用したモデル
model.fit(DifferenceAcc_hist, Hz)

In [43]:
x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, test_size = 0.2, train_size = 0.8, shuffle = True)

In [44]:
# 学習する
clf = SVC()
clf.fit(x_train, y_train)

SVC()

In [45]:
y_pred = clf.predict(x_test)
print("正解率 = ", accuracy_score(y_test, y_pred))

正解率 =  0.0
